# Basic SQWalk commands

In this notebook we will introduce the `SQWalker` class and review its most important features.

After you successfully installed the `sqwalk` package following the [instructions](https://github.com/Buffoni/SQWalk), you are ready to start simulating quantum walkers by importing the `SQWalker` class as follows:

In [ ]:
from sqwalk import SQWalker

Then we import some other libraries that will come handy in the rest of the notebook. As `sqwalk` is based on `QuTiP`, we will make use of some of its functions. In this notebook, we will additionally use `networkx` as a reference package to deal with graphs.

In [ ]:
from qutip import ket2dm, basis, Options, Qobj
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

### Create the graph

First we are going to create the graph on which our walker will move. We will start by defining a simple line graph (called `path_graph` in `networkx`) with $50$ nodes, we will then plot the graph as well as its adjacency matrix.

In [ ]:
#Number of nodes in the line
N = 50
#Build the graph using networkx (or the module of your choice)
graph = nx.path_graph(N)
nx.draw_circular(graph)
plt.show()

In [ ]:
# Plot the adjacency matrix
adj = nx.adj_matrix(graph).todense()
plt.imshow(adj)
plt.show()

### Initialize and run the walker


We have thus associated th graph topology to the adjancency matrix of the graph $A$, whose elements $A_{ij}$ are $1$ is there is a link between the node $i$ and $j$, and $0$ otherwise.
The walker object will then evolve according to the following master equation:

> $ \frac{d \rho}{d t} = (1-p)\ \mathcal{L}_{QW} (\rho) + p\  \mathcal{L}_{CRW} (\rho) + \mathcal{L}_{sink} (\rho)$

where $\mathcal{L}_{QW} (\rho) = - i[A,\rho]$ describes the coherent hoping mechanisms, $\mathcal{L}_{CRW} (\rho) = \sum_{i,j} L_{ij} \rho L_{ij}^{\dagger} - \frac{1}{2} \{L_{ij}^\dagger L_{ij}, \rho\}$ with $L_{ij} = (A_{ij}/d_j)|i><j|$ describes the incoherent hopping ones, while $\mathcal{L}_{sink} (\rho) =  2|n+1><n|\rho |n>< n+1| - \{|n>< n|, \rho\} $ is associated to the irreversible transfer from the graph  (via the node $n$) to the exit (i.e., a sink in the node $n+1$). Besides, $ d_j$ is the number of links attached to the node $j$, while $|i >$ is the element of the basis vectors (in the Hilbert space) corresponding to the node $i$.
The parameter $p$ describes how much incoherent the walker evolution is. In particular, when $p=1$ one recovers the model of a classical random walk, when $p=0$ one faces with a quantum walk, while when $0<p<1$ the walker hops via both incoherent and coherent mechanisms (stochastic quantum walker).


As of now we will leave the sink_node parameter to its default value (None) and we will come back at it later in the notebook.

In [ ]:
# Inizialize the quantum walker that will move on the graph
walker = SQWalker(np.array(adj), noise_param=0.0, sink_node=None)

Now we can simply run the walker by specifying the initial node and the lenght of the evolution. We can time it to see how fast it executes the desired number of steps.

In [ ]:
# Run the walker for 1000 steps starting from the middle node
# time the run to check efficiency
%%time
time_samples = 1000
initial_node = N // 2
result = walker.run_walker(initial_node, time_samples)
new_state = result.final_state
print('Executed', time_samples ,'steps of SQW.')

Note that instead of initializing the initial node of the walker using an integer, we can pass a generic density matrix as initial state of the walker by using qutip.


In [ ]:
%%time
time_samples=1000
initial_quantum_state = ket2dm(basis(N, N//2)) #density matrix equivalent of setting initial_node = N // 2
result = walker.run_walker(initial_quantum_state, time_samples)
new_state = result.final_state
print('Executed', time_samples ,'steps of SQW.')

We can now plot the population in each node by simply taking the diagonal of the final density matrix.

In [ ]:
nodelist = [i for i in range(N)]
plt.bar(nodelist, new_state.diag())
plt.show()

We can then quickly compare this result with an identical classical walker (i.e. setting `noise_param=1`).



In [ ]:
walker = SQWalker(np.array(adj), noise_param=1, sink_node=None)
result = walker.run_walker(initial_node, time_samples)
plt.bar(nodelist, result.final_state.diag())
plt.show()

### Noise and sinks

In general one can pick any value of `noise_param` in the interval $[0,1]$ to get a stochastic quantum walker.

In addition to that we can add a *sink*. But what is it?

A sink is an additional node where the population of the walker is irreversibly stored, we can think of it as a "trap" for our walker or an "exit" from the graph. With `SQWalk` we can specify which node of the graph is connected to the sink via the parameter `sink_node` (multiple sink nodes will come in a following version) which takes as an input the index of the node irreversibly connected to the sink.
Let's pick up again our stochastic quantum walker on a line and attach a sink to the $10th$ node.

In [ ]:
# Inizialize the stochastic quantum walker with a sink connected to the 10th node
walker = SQWalker(np.array(adj), noise_param=0.15, sink_node=10)

opts = Options(store_states=True, store_final_state=True)
result = walker.run_walker(N//2, time_samples=2000, opts=opts)
new_state = result.final_state
result.final_state.shape

We see that now the dimension of the final density matrix is $(51,51)$ instead of $(50,50)$, that is because the sink has been attached to the system. Indeed we can visualize the behavior of the sink by plotting the population on the nodes and see that after some time most of the population is irreversibly in the sink (the last bar).

In [ ]:
nodelist = [i for i in range(N+1)]
plt.bar(nodelist, new_state.diag())
plt.show()

Since the `run_walker` function is based on the `mesolve` solver of QuTip, one can pass some `Options` to the function in the same way in which are passed to `mesolve`.

As you can see above, we asked to store all the indermediate states *and* the final state, in this way we can explore the dynamics of the walker at previous timesteps. Or even use the whole data to create beautiful animations and overlays to the graph.

In [ ]:
# show state at step 1000
plt.bar(nodelist, result.states[1000].diag())
plt.show()

To summarize we have seen how to create a walker from a generic adjacency matrix and reviewed the principal functionalities of `sqwalk` such as the noise parameter, the sink node and the possibility of running time evolutions with different options and initial states.

In the next notebook we will see how one can use an SQWalk object to simulate a walker on a custom and more complex topology class.